# Tugas 3 : Implementasi Algoritma Logistic Regression

In [26]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Memuat Data

In [27]:
df = pd.read_csv("/content/drive/My Drive/ppw/tugas/DataTugas2/Hasil_Prepros.csv")
df.head()

,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,cleansing,case_folding,tokenize,stopword_removal
0,Pesan Anies ke Tom Lembong Tersangka Korupsi I...,Reporter\nNovali Panji Nugroho\nEditor\nEko Ar...,30-10-2024 05:55,Nasional,Reporter\nNovali Panji Nugroho\nEditor\nEko Ar...,reporter\nnovali panji nugroho\neditor\neko ar...,"['reporter', 'novali', 'panji', 'nugroho', 'ed...",reporter novali panji nugroho editor eko ari w...
1,"Tom Lembong Tersangka Korupsi Impor Gula, Nama...",Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,29-10-2024 23:55,Nasional,Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,reporter\nnovali panji nugroho\neditor\nahmad ...,"['reporter', 'novali', 'panji', 'nugroho', 'ed...",reporter novali panji nugroho editor ahmad fai...
2,"Profil Tom Lembong, Eks Mendag dan Co-Captain ...",Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,29-10-2024 22:55,Nasional,Reporter\nNovali Panji Nugroho\nEditor\nAhmad ...,reporter\nnovali panji nugroho\neditor\nahmad ...,"['reporter', 'novali', 'panji', 'nugroho', 'ed...",reporter novali panji nugroho editor ahmad fai...
3,Tom Lembong Sempat Unggah Hal Ini Sehari Sebel...,Reporter\nTempo.co\nEditor\nAndry Triyanto Tji...,30-10-2024 02:55,Nasional,Reporter\nTempoco\nEditor\nAndry Triyanto Tjit...,reporter\ntempoco\neditor\nandry triyanto tjit...,"['reporter', 'tempoco', 'editor', 'andry', 'tr...",reporter tempoco editor andry triyanto tjitra ...
4,"Prabowo Ingin Tingkatkan Pembangunan di Papua,...",Reporter\nVedro Imanuel G\nEditor\nAgung Seday...,29-10-2024 23:55,Bisnis,Reporter\nVedro Imanuel G\nEditor\nAgung Seday...,reporter\nvedro imanuel g\neditor\nagung seday...,"['reporter', 'vedro', 'imanuel', 'g', 'editor'...",reporter vedro imanuel g editor agung sedayu r...


## Import Library

In [28]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


## Pra-pemrosesan Data

In [29]:
# Replace NaN values with empty strings in the 'stopword_removal' column
df['stopword_removal'] = df['stopword_removal'].fillna('')

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Calculate TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

# Convert the result to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)
# Menyimpan hasil ke file CSV
tfidf_df.to_csv('hasil_tfidf.csv', index=False)

In [30]:
# Menggunakan kolom 'cleaned_text' sebagai teks input dan 'kategori' sebagai label
texts = df['stopword_removal'].fillna('')  # Mengganti NaN dengan string kosong jika ada
labels = df['Kategori Berita'].fillna('unknown')  # Mengganti NaN di kategori jika ada

# Melihat label yang digunakan untuk klasifikasi
print(labels.unique())

['Nasional' 'Bisnis']


In [31]:
# Melakukan transformasi TF-IDF
tfidf = TfidfVectorizer(max_features=None)  # Menggunakan 5000 fitur teratas
X_tfidf = tfidf.fit_transform(texts)

# Memisahkan dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Melihat bentuk dari hasil TF-IDF
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (80, 1744)
Shape of X_test: (20, 1744)


## Pelatihan Model

In [32]:
# Membuat model Logistic Regression dengan regularisasi
model = LogisticRegression(max_iter=1000, C=0.3)  # Mengurangi nilai C untuk menambah regularisasi

# Melatih model dengan data yang sudah diresample (jika diperlukan)
model.fit(X_train, y_train)

# Memprediksi hasil pada data testing
y_pred = model.predict(X_test)

## Evaluasi Model

In [33]:
# Evaluasi hasil prediksi
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))

Accuracy: 100.00%
              precision    recall  f1-score   support

      Bisnis       1.00      1.00      1.00        11
    Nasional       1.00      1.00      1.00         9

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [34]:
# Menyimpan tfidf_matrix dan vectorizer menggunakan pickle
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)
# Menyimpan model ke dalam file pickle
with open('logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)

## Prediksi

In [35]:
# Teks baru yang ingin diprediksi
new_text = ['''Jakarta - Bisnis penerbangan di Indonesia diproyeksi melambat kinerja di semester II-2024 ini. Hal ini dikarenakan berbagai faktor seperti besarnya biaya operasional yang diperlukan para maskapai hingga iklim bisnis yang kurang bagus.''']

# Preprocessing dan transformasi menggunakan TF-IDF yang sudah disimpan
new_text_tfidf = tfidf.transform(new_text)

# Melakukan prediksi menggunakan model Logistic Regression
prediction = model.predict(new_text_tfidf)

# Hasil prediksi
predicted_category = "Nasional" if prediction[0] == 'Nasional' else "Bisnis"
print(f"Prediksi: {predicted_category}")

Prediksi: Bisnis


In [36]:
# Mengecek versi scikit-learn
import sklearn
print(sklearn.__version__)

1.5.2


 link streamlit : (https://klasifikasiberitanasionaldanbisnis.streamlit.app/)